In [2]:
import json
import numpy as np
import random
from tqdm import tqdm 

In [3]:
# 数据
train_file = '../data/tmp_data/equal_processed_data/fine45000.txt,../data/tmp_data/equal_processed_data/coarse85000.txt'
val_file = '../data/tmp_data/equal_processed_data/fine5000.txt,../data/tmp_data/equal_processed_data/coarse4588.txt'
neg_attr_dict_file = '../data/tmp_data/equal_processed_data/neg_attr.json'
attr_to_attrvals = '../data/tmp_data/equal_processed_data/attr_to_attrvals.json'
key_attr = "版型"

In [4]:
with open(attr_to_attrvals, 'r') as f:
    attr_to_attrvals = json.load(f)

In [5]:
key_attr_values = attr_to_attrvals[key_attr]
id_to_attr = {}
attr_to_id = {}
for attr_id, attr_v in enumerate(key_attr_values):
    attr_to_id[attr_v] = attr_id
    id_to_attr[attr_id] = attr_v

In [6]:
with open(neg_attr_dict_file, 'r') as f:
    neg_attr_dict = json.load(f)

In [7]:
# 提取数据
def get_data(file_path, key_attr):
    X1 = []
    X2 = []
    Y = []
    i = 0
    for file in file_path.split(','):
        with open(file, 'r') as f:
            for line in tqdm(f):
                item = json.loads(line)
                if item['match']['图文']: # 训练集图文必须匹配
                    if item['key_attr']: # 必须有属性
                        # 生成所有离散属性
                        for key, attr_value in item['key_attr'].items():
                            # 只提取该类别
                            if key == key_attr:
                                # 删除title节省内存
                                X1.append(item["feature"])
                                X2.append(attr_to_id[attr_value])
                                Y.append(1)
                                i+=1
                                # # 生成同类负例
                                # sample_attr_list = neg_attr_dict[attr_value]["similar_attr"]
                                # attr_value = random.sample(sample_attr_list, k=1)[0]
                                # X1.append(item["feature"])
                                # X2.append(attr_to_id[attr_value])
                                # Y.append(0)
                                # i+=1
                # if i >500:
                #     break

    return np.array(X1), np.array(X2), np.array(Y)



In [8]:
X_train1, X_train2, Y_train = get_data(train_file, key_attr)

45000it [00:17, 2624.52it/s]
85000it [00:34, 2493.01it/s]


In [9]:
X_val1, X_val2, Y_val = get_data(val_file, key_attr)

5000it [00:01, 2711.67it/s]
4588it [00:01, 2607.65it/s]


In [10]:
Y_train.shape, Y_val.shape

((53126,), (3804,))

In [13]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train1, X_train2)